In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as ltb
from sklearn.neural_network import MLPClassifier
from copy import deepcopy
import warnings
import math


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

warnings.filterwarnings('ignore')

# Index variables

In [9]:
subject_number = 1

# Find where the static data end from data file 'SXXX_dynamic.csv'

In [10]:
static_end = []

for i in tqdm(range(10,10+subject_number)):
    current_dynamic = pd.read_csv('data_all_intact/S0' + str(i) + '_dynamic.csv', header=None)

    for j in range(len(current_dynamic)-1, -1, -1):
        if current_dynamic.iloc[j,0] == 0:
            static_end.append(j)
            break
    

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
static_end

[5413137]

# Analysis performance for different features

In [12]:
# save accuracy in array 'acc_features'
# save features name in array 'name_features'
acc_features = []
name_features = []

In [13]:
# STD, RMS, IEMG, MAV, WL, LOG, SSI, SKW, KURT, AAC, DASDV

## STD--Standard deviation

In [14]:
# index variable
'''change name, features'''
name = 'STD'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(trainx_raw.iloc[a-385:a+1, b].std())
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(testx_raw.iloc[a-385:a+1,b].std())
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

'change name, features'

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5413137 [00:00<?, ?it/s]

  0%|          | 0/16917 [00:00<?, ?it/s]

  0%|          | 0/5423 [00:00<?, ?it/s]

In [15]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)
trainx_df.to_csv('featured_data/step=50ms/'+name+'_trainx.csv')
trainy_df.to_csv('featured_data/step=50ms/'+name+'_trainy.csv')
testx_df.to_csv('featured_data/step=50ms/'+name+'_testx.csv')
testy_df.to_csv('featured_data/step=50ms/'+name+'_testy.csv')

In [16]:
# train/test model
lightGBM_model = ltb.LGBMClassifier()
lightGBM_model.fit(trainx_df,trainy_df)
expected1 = testy_df.values.flatten()
predicted1 = lightGBM_model.predict(testx_df)
counter = 0
for i in range(0, len(expected1)):
    if expected1[i] == predicted1[i]:
        counter += 1

accuracy1 = counter/len(expected1)
accuracy1

LGBMClassifier()

0.7929190484971418

## RMS--Root Mean Square

In [ ]:
def rms(self):
    current =0
    square =0
    for i in range(0, len(self)):
        current = self.iloc[i]
        square += current ** 2
    mean = square/len(self)
    root = math.sqrt(mean)
    return root

In [ ]:
# index variable
'''change name, features'''
name = 'RMS'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(rms(trainx_raw.iloc[a-385:a+1, b]))
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(rms(testx_raw.iloc[a-385:a+1,b]))
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

In [ ]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)
trainx_df.to_csv('featured_data/step=50ms/'+name+'_trainx.csv')
trainy_df.to_csv('featured_data/step=50ms/'+name+'_trainy.csv')
testx_df.to_csv('featured_data/step=50ms/'+name+'_testx.csv')
testy_df.to_csv('featured_data/step=50ms/'+name+'_testy.csv')

## IEMG--Integrated EMG

In [ ]:
def iemg(self):
    current =0
    absolute =0
    for i in range(0, len(self)):
        current = self.iloc[i]
        absolute += np.abs(current)
    return absolute

In [ ]:
# index variable
'''change name, features'''
name = 'IEMG'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(iemg(trainx_raw.iloc[a-385:a+1, b]))
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(iemg(testx_raw.iloc[a-385:a+1,b]))
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

In [ ]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)
trainx_df.to_csv('featured_data/step=50ms/'+name+'_trainx.csv')
trainy_df.to_csv('featured_data/step=50ms/'+name+'_trainy.csv')
testx_df.to_csv('featured_data/step=50ms/'+name+'_testx.csv')
testy_df.to_csv('featured_data/step=50ms/'+name+'_testy.csv')

## MAV--Mean Absolute Value

In [ ]:
def mav(self):
    current =0
    absolute =0
    for i in range(0, len(self)):
        current = self.iloc[i]
        absolute += np.abs(current)
    return absolute/len(self)

In [ ]:
# index variable
'''change name, features'''
name = 'MAV'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(mav(trainx_raw.iloc[a-385:a+1, b]))
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(mav(testx_raw.iloc[a-385:a+1,b]))
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

In [ ]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)
trainx_df.to_csv('featured_data/step=50ms/'+name+'_trainx.csv')
trainy_df.to_csv('featured_data/step=50ms/'+name+'_trainy.csv')
testx_df.to_csv('featured_data/step=50ms/'+name+'_testx.csv')
testy_df.to_csv('featured_data/step=50ms/'+name+'_testy.csv')

## WL--Waveform Length

In [ ]:
def wl(self):
    current =0
    diff =0
    for i in range(1, len(self)):
        current = self.iloc[i]
        diff += np.abs(current - self.iloc[i-1])
    return diff

In [ ]:
# index variable
'''change name, features'''
name = 'WL'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(wl(trainx_raw.iloc[a-385:a+1, b]))
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(wl(testx_raw.iloc[a-385:a+1,b]))
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

In [ ]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)
trainx_df.to_csv('featured_data/step=50ms/'+name+'_trainx.csv')
trainy_df.to_csv('featured_data/step=50ms/'+name+'_trainy.csv')
testx_df.to_csv('featured_data/step=50ms/'+name+'_testx.csv')
testy_df.to_csv('featured_data/step=50ms/'+name+'_testy.csv')

## LOG--Log Detector

In [ ]:
def log(self):
    current =0
    ln = 0
    for i in range(0, len(self)):
        current = self.iloc[i]
        ln += np.log(np.abs(current))
    return np.exp(ln/len(self))

In [ ]:
# index variable
'''change name, features'''
name = 'LOG'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(log(trainx_raw.iloc[a-385:a+1, b]))
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(log(testx_raw.iloc[a-385:a+1,b]))
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

In [ ]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)
trainx_df.to_csv('featured_data/step=50ms/'+name+'_trainx.csv')
trainy_df.to_csv('featured_data/step=50ms/'+name+'_trainy.csv')
testx_df.to_csv('featured_data/step=50ms/'+name+'_testx.csv')
testy_df.to_csv('featured_data/step=50ms/'+name+'_testy.csv')

## SSI--Simple Square Integral

In [ ]:
def ssi(self):
    current =0
    square = 0
    for i in range(0, len(self)):
        current = self.iloc[i]
        square += current ** 2
    return square

In [ ]:
# index variable
'''change name, features'''
name = 'SSI'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(ssi(trainx_raw.iloc[a-385:a+1, b]))
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(ssi(testx_raw.iloc[a-385:a+1,b]))
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

In [ ]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)
trainx_df.to_csv('featured_data/step=50ms/'+name+'_trainx.csv')
trainy_df.to_csv('featured_data/step=50ms/'+name+'_trainy.csv')
testx_df.to_csv('featured_data/step=50ms/'+name+'_testx.csv')
testy_df.to_csv('featured_data/step=50ms/'+name+'_testy.csv')

## SKW--Skewness

In [ ]:
def skw(self):
    current =0
    result_sum = 0
    for i in range(0, len(self)):
        current = self.iloc[i]
        result_sum += (current-self.mean())**3/len(self)/self.std()**3
    return result_sum

In [ ]:
# index variable
'''change name, features'''
name = 'SKW'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(skw(trainx_raw.iloc[a-385:a+1, b]))
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(skw(testx_raw.iloc[a-385:a+1,b]))
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

In [ ]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)
trainx_df.to_csv('featured_data/step=50ms/'+name+'_trainx.csv')
trainy_df.to_csv('featured_data/step=50ms/'+name+'_trainy.csv')
testx_df.to_csv('featured_data/step=50ms/'+name+'_testx.csv')
testy_df.to_csv('featured_data/step=50ms/'+name+'_testy.csv')

## KURT--Kurtosis

In [ ]:
def kurt(self):
    current =0
    result_sum = 0
    for i in range(0, len(self)):
        current = self.iloc[i]
        result_sum += (current-self.mean())**4/len(self)/self.std()**4
    return result_sum

In [ ]:
# index variable
'''change name, features'''
name = 'KURT'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(kurt(trainx_raw.iloc[a-385:a+1, b]))
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(kurt(testx_raw.iloc[a-385:a+1,b]))
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

In [ ]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)
trainx_df.to_csv('featured_data/step=50ms/'+name+'_trainx.csv')
trainy_df.to_csv('featured_data/step=50ms/'+name+'_trainy.csv')
testx_df.to_csv('featured_data/step=50ms/'+name+'_testx.csv')
testy_df.to_csv('featured_data/step=50ms/'+name+'_testy.csv')

## AAC--Average Amplitude Change

In [ ]:
def aac(self):
    current =0
    diff =0
    for i in range(1, len(self)):
        current = self.iloc[i]
        diff += np.abs(current - self.iloc[i-1])
    return diff/len(self)

In [ ]:
# index variable
'''change name, features'''
name = 'AAC'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(aac(trainx_raw.iloc[a-385:a+1, b]))
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(aac(testx_raw.iloc[a-385:a+1,b]))
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

In [ ]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)
trainx_df.to_csv('featured_data/step=50ms/'+name+'_trainx.csv')
trainy_df.to_csv('featured_data/step=50ms/'+name+'_trainy.csv')
testx_df.to_csv('featured_data/step=50ms/'+name+'_testx.csv')
testy_df.to_csv('featured_data/step=50ms/'+name+'_testy.csv')

## DASDV--Difference Absolute Standard Deviation Value

In [ ]:
def dasdv(self):
    current =0
    square =0
    for i in range(1, len(self)):
        current = self.iloc[i]
        square += (current - self.iloc[i-1])**2
    return square/(len(self)-1)

In [ ]:
# index variable
'''change name, features'''
name = 'DASDV'

# start
for i in tqdm(range(10,10+subject_number)):
    # read csv files
    current_emg = pd.read_csv('data_all_intact/S0' + str(i) + '_emg.csv', header=None)
    current_grasp = pd.read_csv('data_all_intact/S0' + str(i) + '_grasp.csv', header=None)
    current_grasprepetition = pd.read_csv('data_all_intact/S0' + str(i) + '_grasprepetition.csv', header=None)
    
    current_static_emg = current_emg.iloc[0:static_end[i-10]]
    current_static_grasp = current_grasp.iloc[0:static_end[i-10]]
    current_static_grasprepetition = current_grasprepetition.iloc[0:static_end[i-10]]
    
    # find raw train/test data
    train_index = []
    test_index = []
    for j in tqdm(range(len(current_static_grasprepetition))):
        if current_static_grasprepetition.iloc[j,0] in [1,2,3,4,5,6,7,8,9]:
            train_index.append(j)
        if current_static_grasprepetition.iloc[j,0] in [10,11,12]:
            test_index.append(j)
    
    trainx_raw = current_static_emg.iloc[train_index].reset_index(drop=True)
    trainy_raw = current_static_grasp.iloc[train_index].reset_index(drop=True)

    testx_raw = current_static_emg.iloc[test_index].reset_index(drop=True)
    testy_raw = current_static_grasp.iloc[test_index].reset_index(drop=True)
    
    # add overlapping window to train/test data
    ## 385 means the overlapping window is 200ms
    ## train data

    trainx = []
    trainy = []
    
    for a in tqdm(range(385,len(trainx_raw), 96)):
        current_row = []
        for b in range(0,12):
            current_row.append(dasdv(trainx_raw.iloc[a-385:a+1, b]))
        trainx.append(current_row)
        trainy.append(trainy_raw.iloc[a, 0])

    ## test data
    testx = []
    testy = []

    for a in tqdm(range(385,len(testx_raw),96)):
        current_row = []
        for b in range(0,12):
            current_row.append(dasdv(testx_raw.iloc[a-385:a+1,b]))
        testx.append(current_row)
        testy.append(testy_raw.iloc[a, 0])

In [ ]:
trainx_df = pd.DataFrame(trainx)
trainy_df = pd.DataFrame(trainy)
testx_df = pd.DataFrame(testx)
testy_df = pd.DataFrame(testy)
trainx_df.to_csv('featured_data/step=50ms/'+name+'_trainx.csv')
trainy_df.to_csv('featured_data/step=50ms/'+name+'_trainy.csv')
testx_df.to_csv('featured_data/step=50ms/'+name+'_testx.csv')
testy_df.to_csv('featured_data/step=50ms/'+name+'_testy.csv')